In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

conn = sqlite3.connect('/Users/ograndberry/Desktop/berlin_startup_jobs_analysis/bsj_db.db')
c = conn.cursor()

import warnings
warnings.filterwarnings('ignore')

In [3]:
allJobs = pd.read_csv('../data-cleaning/Clean_JobTitles/merged_jobs.csv')
allJobs.head()

,Unnamed: 0,ID,company_name,date,description,jobtitle,source
0,0,1,Fatmap,2017-10-02,Role & Responsibility: \n\nYou’ll be building ...,Mobile Engineer for Outdoor Adventure Sports,Berlin Startup Jobs
1,1,2,AI Engine,2017-10-02,AI Engine is developing innovative machine lea...,Machine Learning,Berlin Startup Jobs
2,2,3,November,2017-10-02,Your mission:\n\nDevelopment of a scalable sof...,Senior Full Stack PHP Developer (f/m),Berlin Startup Jobs
3,3,4,Wayfair,2017-10-02,Your tasks:\n\nKeyword research and competitor...,Jr. Manager SEO UK (Onpage) (m/f),Berlin Startup Jobs
4,4,5,CrossEngage,2017-10-02,About CrossEngage\nCrossEngage is a cloud-base...,Data Scientist,Berlin Startup Jobs


In [4]:
#applied Alina's code to remove non tech jobs
searchfor =['Manager SEO','Salesforce Experte', 'Office Manager', 'Junior Customer Solution', 'Content Management', 'Finance Accountant', 'IT Helpdesk', 'Configuration Manager', 'Distribution Partnerships', 'Digital Marketing Specialist', 'Digital Consultant', 'Quality Assurance Manager', 'Release Manager', 'Strategic Accounts', 'Business Development', 'Physician']
jobTitles = allJobs[~allJobs['jobtitle'].str.contains('|'.join(searchfor))]

In [7]:
def cleaner(data):
    a = data.replace('/', ' ')
    b = a.replace('-', ' ')
    #c = re.sub(r'\([^)]*\)', '', b) #remove unnecessary signs
    d = b.split(' at ')[0] #remove 'at google'
    e = d.split(' for ')[0] #remove 'for google'
    z = e.split('-')[0] #remove 'for google'
    f = z.lower().split()
    sw = [x for x in f if x not in stopwords.words('english')] #remove stopwords
    se = ' '.join(map(str, sw))
    return se

def replace_words(data):
    a = data.replace('jr.', 'junior')
    b = a.replace('sr.', 'senior')
    c = b.replace ('ber','')
    d = c.replace ('berlin','')
    e = d.replace ('fullstack','full stack')
    #z = e.lower().split()
    return e

def intern(data): #return true if any of the following keywords are in the job title
    if 'intern' in data:
        return True
    elif 'working student' in data:
        return True
    elif 'internship' in data:
        return True
    elif 'entry-level' in data:
        return True
    elif 'entry level' in data:
        return True
    else:
        return False 

def seniors(data): #return true if any of the following keywords are in the job title
    if 'senior' in data:
        return True
    elif 'head' in data:
        return True
    elif 'lead' in data:
        return True
    elif 'experienced' in data:
        return True
    elif ' vp ' in data:
        return True
    elif ' cto ' in data:
        return True
    elif 'expert' in data:
        return True
    elif 'director' in data:
        return True
    elif ' sr ' in data:
        return True
    else:
        return False 

def juniors(data): #return true if any of the following keywords are in the job title
    if 'junior' in data:
        return True
    elif 'entry-level' in data:
        return True
    elif 'entry level' in data:
        return True
    elif 'graduate' in data:
        return True
    else:
        return False

def intermediate(data): #return true if any of the following keywords are in the job title
    if 'intermediate' in data:
        return True
    elif 'mid level' in data:
        return True
    elif 'mid-level' in data:
        return True
    elif ' mid ' in data:
        return True
    else:
        return False 

def removeSigns(data):
    c = re.sub(r'\([^)]*\)', '', data) #remove unnecessary signs
    return c

In [8]:
jobTitles['NewJobTitles'] = jobTitles['jobtitle'].apply(cleaner)
jobTitles['jobtitle'] = jobTitles['NewJobTitles'].apply(replace_words)
jobTitles['NewJobTitles'] = jobTitles['NewJobTitles'].apply(removeSigns)

jobTitles["Intern"] = jobTitles["jobtitle"].apply(intern)
jobTitles["Junior"] = jobTitles["jobtitle"].apply(juniors)
jobTitles["Intermediate"] = jobTitles["jobtitle"].apply(intermediate)
jobTitles["Senior"] = jobTitles["jobtitle"].apply(seniors)


In [9]:
positionlevel = pd.DataFrame(jobTitles,columns=['ID','Intern','Junior','Intermediate','Senior'])

In [11]:
positionlevel = positionlevel.set_index('ID') #position level 
positionlevel.head()

,Intern,Junior,Intermediate,Senior
ID,,,,
1,False,False,False,False
2,False,False,False,False
3,False,False,False,True
5,False,False,False,False
6,False,False,False,False


In [12]:
positionlevel.to_sql('Level', conn, if_exists='replace')